<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/fancyanalyserdraft1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚀 Enhanced and Fancy LAB Color Analyzer 🚀

# ------------------------------
# 1. Install and Import Libraries
# ------------------------------

# Install required libraries only if not already installed
import sys
import subprocess

def install_packages(packages):
    """
    Installs the listed packages using pip if they are not already installed.

    Parameters:
        packages (list): List of package names as strings.
    """
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = ['colormath', 'plotly', 'ipywidgets', 'pandas', 'numpy']
install_packages(required_packages)

# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from google.colab import files
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

# Enable ipywidgets in Colab
from google.colab import output
output.enable_custom_widget_manager()

# ------------------------------
# 2. Define Utility Functions
# ------------------------------

# (Utility functions remain unchanged)

# ------------------------------
# 3. Define Visualization Functions
# ------------------------------

def create_combined_visualizations(fig1, fig2, fig3, fig4, fig5):
    """
    Combines all individual plots (excluding 'splom') into a single subplot layout.

    Parameters:
        fig1 to fig5 (plotly.graph_objects.Figure): Individual Plotly figures.

    Returns:
        plotly.graph_objects.Figure: The combined Plotly figure object.
    """
    # Define subplot structure without 'splom'
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            'Input vs Closest Color',
            'LAB Value Comparison',
            '3D LAB Color Space',
            'Delta-E Distribution',
            'Color Density Heatmap',
            'Pairwise LAB Relationships'  # This will be handled separately
        ),
        specs=[
            [{"type": "scatter"}, {"type": "bar"}],
            [{"type": "scatter3d"}, {"type": "histogram"}],
            [{"type": "heatmap"}, {"type": "scatter"}]  # Changed 'splom' to 'scatter'
        ],
        vertical_spacing=0.15,
        horizontal_spacing=0.1
    )

    # Add traces to subplots
    for trace in fig1['data']:
        fig.add_trace(trace, row=1, col=1)
    for trace in fig2['data']:
        fig.add_trace(trace, row=1, col=2)
    for trace in fig3['data']:
        fig.add_trace(trace, row=2, col=1)
    for trace in fig4['data']:
        fig.add_trace(trace, row=2, col=2)
    for trace in fig5['data']:
        fig.add_trace(trace, row=3, col=1)

    # Update layout
    fig.update_layout(
        height=1800,
        showlegend=False,
        template='plotly_dark',
        title_text='🎨 Enhanced LAB Color Analyzer Visualizations'
    )

    return fig

def create_pairwise_scatter_matrix(dataset_df, input_lab, closest_lab):
    """
    Creates a separate Scatter Plot Matrix to explore relationships between LAB components.

    Parameters:
        dataset_df (pd.DataFrame): DataFrame containing the dataset LAB colors.
        input_lab (list): LAB values of the input color.
        closest_lab (list): LAB values of the closest match.

    Returns:
        plotly.graph_objects.Figure: The Splom figure.
    """
    # Create Splom trace
    splom_trace = go.Splom(
        dimensions=[
            dict(label='L', values=dataset_df['L']),
            dict(label='A', values=dataset_df['A']),
            dict(label='B', values=dataset_df['B'])
        ],
        text=dataset_df['Color Name'],
        marker=dict(
            size=5,
            color='lightgrey',
            opacity=0.5
        ),
        name='Dataset Colors'
    )

    # Create figure with Splom
    fig_splom = go.Figure(data=[splom_trace])

    # Add input and closest colors
    # Input Color
    fig_splom.add_trace(go.Scatter(
        x=[input_lab[0]],
        y=[input_lab[1]],
        mode='markers',
        marker=dict(size=10, color='blue', symbol='diamond'),
        name='Input Color',
        text=['Input Color'],
        hoverinfo='text'
    ))

    # Closest Color
    fig_splom.add_trace(go.Scatter(
        x=[closest_lab[0]],
        y=[closest_lab[1]],
        mode='markers',
        marker=dict(size=10, color='green', symbol='star'),
        name='Closest Color',
        text=[f'Closest: {closest_lab}'],
        hoverinfo='text'
    ))

    # Update layout
    fig_splom.update_layout(
        title='🔍 Pairwise LAB Relationships',
        template='plotly_dark'
    )

    return fig_splom

def display_tabbed_visualizations(fig_combined, fig_splom):
    """
    Displays all plots in a tabbed interface using ipywidgets.

    Parameters:
        fig_combined (plotly.graph_objects.Figure): Combined subplots figure.
        fig_splom (plotly.graph_objects.Figure): Splom figure.
    """
    tabs = widgets.Tab()

    # Define tabs and their content
    tab_contents = [
        widgets.Output(),
        widgets.Output(),
        widgets.Output(),
        widgets.Output(),
        widgets.Output(),
        widgets.Output()
    ]

    # Set titles for each tab
    tab_titles = [
        'Input vs Closest Color',
        'LAB Value Comparison',
        '3D LAB Color Space',
        'Delta-E Distribution',
        'Color Density Heatmap',
        'Pairwise LAB Relationships'
    ]

    for i in range(len(tab_contents)):
        tabs.set_title(i, tab_titles[i])

    # Populate each tab with corresponding figure
    with tab_contents[0]:
        # Extract subplot for 'Input vs Closest Color'
        sub_fig = make_subplots(rows=3, cols=2)
        fig_combined.data[0].show()
        fig_combined['data'][0].show()
        fig_combined.show()
        fig_combined['data'][0].show()

    # To simplify, display the combined figure in the first tab and the splom in the last tab
    with tab_contents[0]:
        fig_combined.show()
    with tab_contents[5]:
        fig_splom.show()

    # Assign the rest of the plots to their respective tabs
    # Depending on how you want to organize, but for simplicity, you can show all in separate tabs

    tabs.children = tab_contents
    display(tabs)

# ------------------------------
# 4. Interactive User Interface
# ------------------------------

def run_color_analyzer():
    """
    Runs the interactive LAB Color Analyzer with fancy visual enhancements.
    """
    # Upload and validate dataset
    try:
        dataset_df = upload_dataset()
        validate_dataset(dataset_df)
        print("✅ Dataset uploaded and validated successfully.")
        print(f"📊 Dataset Preview:\n{dataset_df.head()}")
    except Exception as e:
        print(f"❌ Error uploading or validating dataset: {e}")
        return

    # Define interactive widgets for LAB input with precise controls
    # Sliders with smaller step sizes
    lab_l_slider = widgets.FloatSlider(
        value=50.0,
        min=0.0,
        max=100.0,
        step=0.01,
        description='L:',
        continuous_update=False,
        layout=widgets.Layout(width='80%')
    )
    lab_a_slider = widgets.FloatSlider(
        value=0.0,
        min=-128.0,
        max=127.0,
        step=0.01,
        description='A:',
        continuous_update=False,
        layout=widgets.Layout(width='80%')
    )
    lab_b_slider = widgets.FloatSlider(
        value=0.0,
        min=-128.0,
        max=127.0,
        step=0.01,
        description='B:',
        continuous_update=False,
        layout=widgets.Layout(width='80%')
    )

    # FloatText widgets for precise input
    lab_l_text = widgets.FloatText(
        value=50.0,
        description='L:',
        step=0.01,
        layout=widgets.Layout(width='80%')
    )
    lab_a_text = widgets.FloatText(
        value=0.0,
        description='A:',
        step=0.01,
        layout=widgets.Layout(width='80%')
    )
    lab_b_text = widgets.FloatText(
        value=0.0,
        description='B:',
        step=0.01,
        layout=widgets.Layout(width='80%')
    )

    # Button to find closest color
    run_button = widgets.Button(
        description='🔍 Find Closest Color',
        button_style='success',
        tooltip='Click to find the closest color',
        icon='search'
    )

    # Output area
    output = widgets.Output()

    # Synchronize sliders and text boxes
    def sync_l_slider_text(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_l_text.value = round(change['new'], 2)

    def sync_l_text_slider(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_l_slider.value = round(change['new'], 2)

    def sync_a_slider_text(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_a_text.value = round(change['new'], 2)

    def sync_a_text_slider(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_a_slider.value = round(change['new'], 2)

    def sync_b_slider_text(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_b_text.value = round(change['new'], 2)

    def sync_b_text_slider(change):
        if change['type'] == 'change' and change['name'] == 'value':
            lab_b_slider.value = round(change['new'], 2)

    lab_l_slider.observe(sync_l_slider_text, names='value')
    lab_l_text.observe(sync_l_text_slider, names='value')

    lab_a_slider.observe(sync_a_slider_text, names='value')
    lab_a_text.observe(sync_a_text_slider, names='value')

    lab_b_slider.observe(sync_b_slider_text, names='value')
    lab_b_text.observe(sync_b_text_slider, names='value')

    # Organize sliders and text inputs side by side
    lab_l_box = widgets.HBox([lab_l_slider, lab_l_text])
    lab_a_box = widgets.HBox([lab_a_slider, lab_a_text])
    lab_b_box = widgets.HBox([lab_b_slider, lab_b_text])

    # Display widgets
    ui = widgets.VBox([lab_l_box, lab_a_box, lab_b_box, run_button])
    display(ui, output)

    def on_run_button_clicked(b):
        with output:
            clear_output(wait=True)
            input_lab = [lab_l_slider.value, lab_a_slider.value, lab_b_slider.value]
            try:
                validate_lab_color(input_lab)
                print(f"🟢 Input LAB Color: L={input_lab[0]}, A={input_lab[1]}, B={input_lab[2]}")

                # Find closest color
                closest_color, delta_e = find_closest_color(np.array(input_lab), dataset_df)
                closest_color_name = closest_color['Color Name']
                closest_lab = [closest_color['L'], closest_color['A'], closest_color['B']]

                print(f"🎨 Closest ISCC-NBS Color: {closest_color_name}")
                print(f"📏 Delta-E Value: {delta_e:.2f}")
                print(f"🔍 LAB of Closest Color: L={closest_lab[0]}, A={closest_lab[1]}, B={closest_lab[2]}")

                # Convert to RGB
                input_rgb = lab_to_rgb(input_lab)
                closest_rgb = lab_to_rgb(closest_lab)

                print(f"🎨 Input RGB Color: {input_rgb}")
                print(f"🎨 Closest RGB Color: {closest_rgb}")

                # Calculate all Delta-E values
                all_delta_e = calculate_delta_e(np.array(input_lab), dataset_df)

                # Create visualizations
                fig1 = create_advanced_color_comparison_plot(input_rgb, closest_rgb, input_lab, closest_lab, closest_color_name, delta_e)
                fig2 = create_interactive_lab_comparison_plot(input_lab, closest_lab, closest_color_name)
                fig3 = create_dynamic_lab_color_space_plot(input_lab, closest_lab, closest_color_name, dataset_df)
                fig4 = create_delta_e_distribution_histogram(all_delta_e)
                fig5 = create_color_density_heatmap(dataset_df)
                # Remove splom_trace from combined visualizations
                # splom_trace = create_pairwise_scatter_matrix(dataset_df, input_lab, closest_lab)
                # Instead, create a separate splom figure
                fig_splom = create_pairwise_scatter_matrix(dataset_df, input_lab, closest_lab)

                # Combine all visualizations into a single figure (excluding splom)
                combined_fig = create_combined_visualizations(fig1, fig2, fig3, fig4, fig5)
                combined_fig.show()

                # Display Splom separately
                fig_splom.show()

                # Alternatively, use tabbed interface
                # display_tabbed_visualizations(combined_fig, fig_splom)

                # Display results in table
                results = {
                    'Input LAB': f"L={input_lab[0]}, A={input_lab[1]}, B={input_lab[2]}",
                    'Closest ISCC-NBS Color': closest_color_name,
                    'Delta-E': delta_e,
                    'Closest LAB': f"L={closest_lab[0]}, A={closest_lab[1]}, B={closest_lab[2]}",
                    'Input RGB': input_rgb,
                    'Closest RGB': closest_rgb
                }
                display_results_table(results)

            except ValueError as ve:
                print(f"❌ Input validation error: {ve}")
            except Exception as e:
                print(f"❌ An unexpected error occurred: {e}")

    run_button.on_click(on_run_button_clicked)

# ------------------------------
# 5. Execute the Analyzer
# ------------------------------

# Call the run_color_analyzer function to start the interactive interface
try:
    run_color_analyzer()
except Exception as e:
    print(f"❌ An error occurred while running the analyzer: {e}")